In [16]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine

# Charger le fichier .env
load_dotenv(dotenv_path=".env")  # ou spécifie le chemin absolu si besoin

# Lire les variables
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")

# Construire l'URL PostgreSQL
url = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Créer l'engine SQLAlchemy
engine = create_engine(url)


In [17]:
with engine.connect() as conn:
    test = pd.read_sql("SELECT 1", conn)
test


,?column?
0,1


In [18]:
#  Récupérer les 10 premières lignes de la table translations

with engine.connect() as conn:
    df = pd.read_sql("SELECT * FROM translations LIMIT 10", conn)
df


,id,source_lang,source_text,target_lang,target_text
0,1,fr,Tu connais un bon restaurant marocain dans le ...,dr,تعرف شي مطعم مغربي زوين فهاد الناحية؟
1,2,fr,Quel plat typique dois-je absolument goûter ?,dr,واش من طبق تقليدي خصني بالضرورة نذوقو؟
2,3,fr,Où trouver le meilleur couscous ici ?,dr,فين نلقى أحسن كسكس هنا؟
3,4,fr,C'est quoi ta pâtisserie préférée marocaine ?,dr,علاش هي حلويات المغربية المفضلة عندك؟
4,5,fr,Le thé à la menthe se boit à quel moment génér...,dr,الأتاي بالنعناع يتشرب فوقاش فالعادة؟
5,6,fr,Je voudrais essayer un tajine authentique.,dr,بغيت نجرب طاجين أصلي.
6,7,fr,Tu préfères manger dans les restos ou la stree...,dr,تفضل تاكل في المطاعم ولا في الأكل ديال الشارع؟
7,8,fr,Tu connais une spécialité locale peu connue ?,dr,كتعرف شي وجبة محلية ما معروفاش بزاف؟
8,9,fr,J'aimerais goûter des plats végétariens maroca...,dr,بغيت نجرب شي ماكلات مغربية نباتية.
9,10,fr,Quels fruits locaux je devrais absolument essa...,dr,أش من فواكه محلية خصني نجرب؟


In [19]:
# Obtenir le nombre total de lignes dans la table translations
with engine.connect() as conn:
    count = pd.read_sql("SELECT COUNT(*) FROM translations", conn)
count


,count
0,51903


In [20]:
# 📊Regrouper les traductions par source_lang et target_lang
with engine.connect() as conn:
    stats = pd.read_sql("""
        SELECT source_lang, target_lang, COUNT(*) as total
        FROM translations
        GROUP BY source_lang, target_lang
        ORDER BY total DESC
    """, conn)
stats


,source_lang,target_lang,total
0,en,dr,15520
1,dr,en,15513
2,fr,dr,10949
3,dr,fr,9921


In [21]:
# 🔁 Chercher des doublons dans les traductions (mêmes texte et langues)
with engine.connect() as conn:
    duplicates = pd.read_sql("""
        SELECT source_lang, source_text, target_lang, COUNT(*) as occurrences
        FROM translations
        GROUP BY source_lang, source_text, target_lang
        HAVING COUNT(*) > 1
    """, conn)
duplicates


,source_lang,source_text,target_lang,occurrences
0,dr,سير نيشان,fr,2
1,dr,واعر,en,2
2,fr,"Eh bien, je ne trouve pas la carte",dr,2
3,fr,Pas si bien.,dr,2
4,en,"So, how has your week been?",dr,2
...,...,...,...,...
912,en,how are things going,dr,2
913,fr,Je t'aime,dr,2
914,en,Don't get upset!,dr,2
915,fr,Où est la poste ?,dr,2
